# async module 스터디 리뷰
- 날짜: 2024/01/11

### 테스트 코드 작성 과정
1. 최종 콜백 사용 유무
    - 최종 콜백을 사용하는 방법 (기본 예제)
    - 최종 콜백을 사용하지 않는 방법 (async/await, try/catch 사용)

2. 케이스 분기
    - 정상 작동 케이스
    - 중간 콜백이 하나라도 호출되지 않는 케이스
    - 비동기 작업 중 에러를 발생시키는 케이스

3. 순차적, 병렬적 메소드 작동 방식 분석
    - waterfall vs parallel

### 학습한 메소드
- collections
    1. each
    2. filter
    3. groupBy
    4. map
    5. reduce
    6. some
- controlFlow
    1. parallel
    2. retry
    3. until
    4. waterfall

-------------------
## 정상 작동 케이스

In [1]:
const async = require('async');

collection - each()

In [2]:
// 정상 작동하는 테스트 케이스
// 모든 비동기 함수에서 콜백이 호출되어 최종 콜백이 정상적으로 호출된다.

function asyncFunction(item, callback) {
    console.log('처리 중인 Item: ', item);
    setTimeout(()=>{
        console.log('처리 완료된 Item: ', item);
        callback(null); 
    }, item * 1000);
}

function finalCallback (err) {
    if(err) console.error('오류 발생', err);
    else console.log('모든 항목 처리 완료');
}

async.each([3, 2, 1], asyncFunction, finalCallback);

처리 중인 Item:  3
처리 중인 Item:  2
처리 중인 Item:  1


처리 완료된 Item:  1
처리 완료된 Item:  2
처리 완료된 Item:  3
모든 항목 처리 완료


controlFlow - parallel()

In [ ]:
// tasks가 병렬적으로 실행된다.
// results에는 각 비동기 함수의 수행 시간과 상관없이 task의 순서대로 결과값이 담긴다.

// 비동기 작업들
function asyncFirstFunction(callback) {
    setTimeout(() => {
        console.log('첫 번째 처리 중')
        callback(null, '첫 번째 ');
    }, 3000);
};
function asyncSecondFunction (callback) {
    setTimeout(() => {
        console.log('두 번째 처리 중')
        callback(null, '두 번째 ');
    }, 1000);
};
function asyncThirdFunction(callback) {
    setTimeout(() => {
        console.log('세 번째 처리 중')
        callback(null, '세 번째 ');
    }, 2000);
};

// 최종 콜백(opt)
function finalCallback(err, result) {
    if (err) console.error('오류 발생', err);
    else console.log('최종 결과:', result);
};

async.parallel([asyncFirstFunction, asyncSecondFunction, asyncThirdFunction], finalCallback);

두 번째 처리 중
세 번째 처리 중
첫 번째 처리 중
최종 결과: [ '첫 번째 ', '두 번째 ', '세 번째 ' ]


--------------
## 중간 콜백을 호출하지 않는 케이스

collections - filter()

In [4]:
// 두 번째 요소가 콜백을 호출하지 않아 최종 콜백도 실행되지 않는다.

// 각 요소에 대해 수행할 비동기 작업
function asyncFunction(item, callback) {
    console.log('처리 중인 Item: ', item);
    setTimeout(() => {
        console.log('처리 완료된 Item: ', item);
        if(item !== 2) callback(null, item > 2); 
    }, item * 1000);
};

// 최종 콜백
function finalCallback (err, result) {
    if (err) console.error('오류 발생', err);
    else console.log('조건을 만족하는 요소', result);
};

async.filter([4, 3, 2, 1], asyncFunction, finalCallback);

처리 중인 Item:  4
처리 중인 Item:  3
처리 중인 Item:  2
처리 중인 Item:  1


처리 완료된 Item:  1
처리 완료된 Item:  2
처리 완료된 Item:  3
처리 완료된 Item:  4


controlFlow - retry()

In [11]:
// 중간 콜백이 실행되지 않으면, 다음 작업으로 넘어가지 않는다.

// 30% 확률로 성공하는 비동기 작업
function asyncFunction (callback) {
    setTimeout(() => {
        console.log('시도!');
        const result = Math.random() > 0.7;
        if (result) {
            console.log('성공...');
            // callback(null, '성공!');
        } else {
            console.log('실패...');
            // callback('실패');
        }
    }, 1000);
};

// 최종 콜백(opt)
function finalCallback (err, result) {
    if (err) console.error('오류 메시지: ', err);
    else console.log('작업 결과', result);
};

async.retry({ times: 3, interval: 200 }, asyncFunction, finalCallback);

시도!
실패...


----------
## 비동기 작업 중 에러를 발생시키는 케이스

collections - each()

In [12]:
// 비동기 작업 중 에러가 발생하는 케이스
// 비동기 작업 중 에러를 발생시키면, 에러가 최종 콜백으로 넘겨진다.

function asyncFunction(item, callback) {
    console.log('처리 중인 Item: ', item);
    setTimeout(()=>{
        console.log('처리 완료된 Item: ', item);
        if(item === 2) callback(`${item}에서 에러 발생`);
        else callback(null);
    }, item * 1000);
}

function finalCallback (err) {
    if(err) console.error('오류 메시지:', err);
    else console.log('모든 항목 처리 완료');
}

async.each([3, 2, 1], asyncFunction, finalCallback);

처리 중인 Item:  3
처리 중인 Item:  2
처리 중인 Item:  1


처리 완료된 Item:  1
처리 완료된 Item:  2


오류 메시지: 2에서 에러 발생


처리 완료된 Item:  3


controlFlow - waterfall()

In [13]:
// 두 번째 비동기 함수에서 에러 발생
// 에러 발생 직후, 최종 콜백에서 에러 핸들링

// 비동기 작업들
function asyncFirstFunction(callback) {
    setTimeout(() => {
        console.log('첫 번째 처리 중')
        callback(null, '첫 번째 ');
    }, 3000);
};
function asyncSecondFunction (arg, callback) {
    setTimeout(() => {
        console.log('두 번째 처리 중')
        callback('두 번째 함수에서 오류 발생');
    }, 1000);
};
function asyncThirdFunction(arg, callback) {
    setTimeout(() => {
        console.log('세 번째 처리 중')
        callback(null, arg + '세 번째 ');
    }, 2000);
};

// 최종 콜백(opt)
function finalCallback(err, result) {
    if (err) console.error('오류 발생 메시지: ', err);
    else console.log('최종 결과:', result);
};

async.waterfall([asyncFirstFunction, asyncSecondFunction, asyncThirdFunction], finalCallback);

첫 번째 처리 중
두 번째 처리 중


오류 발생 메시지:  두 번째 함수에서 오류 발생


=> 비동기 함수가 병렬적으로 실행되면, 최종 콜백이 호출되더라도 다른 비동기 작업들이 계속 진행된다.

=> 비동기 함수가 순차적으로 실행되면, 최종 콜백이 호출되면 다음 비동기 작업이 수행되지 않는다.

--------
## 최종 콜백을 사용하지 않는 방법

controlFlow - waterfall()

async 라이브러리의 index.ts => async.waterfall

```javascript
export function waterfall<T>(tasks: Function[]): Promise<T>;
export function waterfall<T, E = Error>(tasks: Function[], callback: AsyncResultCallback<T, E>): void;
```

-> callback 파라미터를 사용하지 않으면, Promise를 반환한다.

-> 따라서 최종 콜백을 사용하지 않으면 반환되는 Promise를 처리해주면 된다.

In [14]:
// async, await 사용
// 비동기 작업들
function asyncFirstFunction(callback) {
    setTimeout(() => {
        console.log('첫 번째 처리 중')
        callback(null, '첫 번째 ');
    }, 3000);
};
function asyncSecondFunction (arg, callback) {
    setTimeout(() => {
        console.log('두 번째 처리 중')
        callback(null, arg + '두 번째 ');
    }, 1000);
};
function asyncThirdFunction(arg, callback) {
    setTimeout(() => {
        console.log('세 번째 처리 중')
        callback(null, arg + '세 번째 ');
    }, 2000);
};

async function runWaterfallTasks(){
    try {
        const result = await async.waterfall([
            asyncFirstFunction, asyncSecondFunction, asyncThirdFunction
        ]);
        console.log('모든 작업 완료:', result);
    } catch (err) {
        console.error('에러 발생: ', err);
    }
}

runWaterfallTasks();

Promise { <pending> }

첫 번째 처리 중
두 번째 처리 중
세 번째 처리 중
모든 작업 완료: 첫 번째 두 번째 세 번째 


In [15]:
// then, catch 사용
// 비동기 작업들
function asyncFirstFunction(callback) {
    setTimeout(() => {
        console.log('첫 번째 처리 중')
        callback(null, '첫 번째 ');
    }, 3000);
};
function asyncSecondFunction (arg, callback) {
    setTimeout(() => {
        console.log('두 번째 처리 중')
        callback(null, arg + '두 번째 ');
    }, 1000);
};
function asyncThirdFunction(arg, callback) {
    setTimeout(() => {
        console.log('세 번째 처리 중')
        callback(null, arg + '세 번째 ');
    }, 2000);
};

async.waterfall([asyncFirstFunction, asyncSecondFunction, asyncThirdFunction]).then((result) => {
    console.log('모든 작업 완료:', result);
 }).catch((err) => console.error('에러 메시지:', err));       

Promise { <pending> }

첫 번째 처리 중
두 번째 처리 중
세 번째 처리 중
모든 작업 완료: 첫 번째 두 번째 세 번째 


waterfall(순차적) vs parallel(병렬적) 내부적 작동 방식

waterfall.js
```javascript
function waterfall (tasks, callback) {
    callback = once(callback);
    if (!Array.isArray(tasks)) return callback(new Error('First argument to waterfall must be an array of functions'));
    if (!tasks.length) return callback();
    var taskIndex = 0;
    function nextTask(args) {
        var task = wrapAsync(tasks[taskIndex++]);
        task(...args, onlyOnce(next));
    }
    function next(err, ...args) {
        if (err === false) return
        if (err || taskIndex === tasks.length) {
            return callback(err, ...args);
        }
        nextTask(args);
    }
    nextTask([]);
}
export default awaitify(waterfall)
```

1. task를 wrapAsync로 감싸서 동기식으로 호출할 수 있도록 하고
2. task가 완료되면 next를 호출
3. next에서는 nextTask를 호출하며 arg를 넘겨준다.
4. 만약 next에서 마지막 task를 처리했으면 최종 callback을 return

parallel.js
```javascript
exports.default = (0, _awaitify2.default)((eachfn, tasks, callback) => {
    var results = (0, _isArrayLike2.default)(tasks) ? [] : {};

    eachfn(tasks, (task, key, taskCb) => {
        (0, _wrapAsync2.default)(task)((err, ...result) => {
            if (result.length < 2) {
                [result] = result;
            }
            results[key] = result;
            taskCb(err);
        });
    }, err => callback(err, results));
}, 3);
module.exports = exports.default;
```
1. eachfn으로 각 task에 대해 실행
2. task를 wrapAsync로 감싸 비동기적으로 처리
3. results[key] = result를 통해 task의 순서에 맞게 결과를 저장
4. 각 작업이 완료되면 taskCb(taskCallback) 호출
5. 최종적으로 모든 작업이 완료되거나 에러가 발생하면 callback(err, results) 호출


### 질문
Async 함수를 구현할 때 주로 사용되는 두 함수, wrapAsync와 awaitify의 차이
 